## Word Embeddings con Word2Vec

- Entrenar modelos Word2Vec con CBOW y Skip-Gram
- Buscar palabras similares
- Visualizar embeddings en 2D usando t-SNE y PCA
- Mostrar noticias de diferentes tópicos en el espacio de embeddings

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from gensim.models import Word2Vec
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

In [2]:
df = pd.read_csv(
    "noticias_unificadas.tsv",
    encoding="utf-8",
    sep="\t",
    dtype={"fecha": "string", "titulo": "string", "contenido": "string", "seccion": "string", "link": "string"},
    quoting=0,
    na_filter=False
)

print(f"Total de documentos: {len(df)}")
print(f"\nCategorías: {df['seccion'].nunique()}")
print(df['seccion'].value_counts())

Total de documentos: 37746

Categorías: 7
seccion
Política        12509
Espectáculos     6386
Mundo            5186
Deportes         4739
Cultura          3256
Economía         3168
Policiales       2502
Name: count, dtype: Int64


In [4]:
from utils.utils import clean_text

df["headline_text"] = (df["titulo"].fillna("") + " " + df["contenido"].fillna(""))


In [7]:
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
import gensim

# Stopwords en español
STOPWORDS = set(stopwords.words("spanish"))

STOP_EXTRA = {
    "dijo", "anos", "foto", "video", "puedes", "ver", "hoy", "ayer", "manana", 
    "mas", "recomendado", "ser", "dia", "dias", "tambien", "cada", "tras", 
    "soles", "uno", "dos", "tres", "asi", "mil", "ano", "año", "solo", 
    "senalo", "segun", "entre", "millones", "lugar", "puede", "haber", 
    "tener", "sol", "precio", "yape", "pai", "nueva", "hace", "hacer"
}

STOPWORDS |= STOP_EXTRA

print(f"Total de stopwords: {len(STOPWORDS)}")

Total de stopwords: 352


In [8]:

def preprocess_for_w2v(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in STOPWORDS and len(token) > 3:
            result.append(token)
    return result


ejemplo = "El presidente anunció nuevas elecciones políticas en el congreso"
print("Texto original:")
print(ejemplo)
print("\nTokens procesados:")
print(preprocess_for_w2v(ejemplo))

Texto original:
El presidente anunció nuevas elecciones políticas en el congreso

Tokens procesados:
['presidente', 'anunció', 'nuevas', 'elecciones', 'políticas', 'congreso']


In [9]:

print("Procesando documentos para Word2Vec...")
processed_docs = df['headline_text'].map(preprocess_for_w2v)

sentences = processed_docs.tolist()

print(f"\nTotal de documentos procesados: {len(sentences)}")
print(f"\nEjemplo de documento procesado:")
print(sentences[100][:20])

doc_lengths = [len(doc) for doc in sentences]
print(f"\nEstadísticas de longitud:")
print(f"  Media: {np.mean(doc_lengths):.2f} tokens")
print(f"  Mediana: {np.median(doc_lengths):.2f} tokens")
print(f"  Min: {min(doc_lengths)}, Max: {max(doc_lengths)}")

Procesando documentos para Word2Vec...

Total de documentos procesados: 37746

Ejemplo de documento procesado:
['diego', 'leon', 'director', 'cine', 'siempre', 'quisimos', 'contar', 'hazana', 'militar', 'entrevista', 'diego', 'leon', 'toma', 'calma', 'exito', 'polemica', 'generado', 'primera', 'semana', 'exhibicion']

Estadísticas de longitud:
  Media: 195.71 tokens
  Mediana: 166.00 tokens
  Min: 13, Max: 6543


## Entrenar Modelo Word2Vec

### CBOW (Continuous Bag of Words)
- Predice la palabra central dado el contexto
- Más rápido de entrenar
- Mejor para palabras frecuentes

### Skip-Gram
- Predice el contexto dada la palabra central
- Más lento de entrenar
- Mejor para palabras poco frecuentes

In [ ]:
print("ENTRENANDO WORD2VEC CON CBOW")

vector_size = 100
window = 5
min_count = 5
workers = 4
epochs = 20

# Entrenar CBOW (sg=0)
model_cbow = Word2Vec(
    sentences=sentences,
    vector_size=vector_size,
    window=window,
    min_count=min_count,
    workers=workers,
    epochs=epochs,
    sg=0,
    seed=42
)

print(f"\n✅ Modelo CBOW entrenado exitosamente")
print(f"Vocabulario: {len(model_cbow.wv)} palabras")

ENTRENANDO WORD2VEC CON CBOW


In [ ]:
print("ENTRENANDO WORD2VEC CON SKIP-GRAM")

# Entrenar Skip-Gram (sg=1)
model_skipgram = Word2Vec(
    sentences=sentences,
    vector_size=vector_size,
    window=window,
    min_count=min_count,
    workers=workers,
    epochs=epochs,
    sg=1,
    seed=42
)

print(f"\n✅ Modelo Skip-Gram entrenado exitosamente")
print(f"Vocabulario: {len(model_skipgram.wv)} palabras")

## 🔍 Buscar Palabras Similares

In [ ]:
# CELDA 9: Función para buscar palabras similares
def find_similar_words(model, word, topn=10):
    """
    Encuentra palabras similares usando el modelo Word2Vec
    """
    try:
        similar = model.wv.most_similar(word, topn=topn)
        return similar
    except KeyError:
        return None

def compare_models_similarity(word, model_cbow, model_skipgram, topn=10):
    """
    Compara resultados de similitud entre CBOW y Skip-Gram
    """
    print("=" * 80)
    print(f"PALABRAS SIMILARES A: '{word}'")
    print("=" * 80)
    
    # CBOW
    print(f"\n🔹 Modelo CBOW:")
    similar_cbow = find_similar_words(model_cbow, word, topn)
    if similar_cbow:
        for i, (w, score) in enumerate(similar_cbow, 1):
            print(f"  {i:2}. {w:20s} → similitud: {score:.4f}")
    else:
        print(f"  ⚠️ Palabra '{word}' no encontrada en vocabulario CBOW")
    
    # Skip-Gram
    print(f"\n🔹 Modelo Skip-Gram:")
    similar_sg = find_similar_words(model_skipgram, word, topn)
    if similar_sg:
        for i, (w, score) in enumerate(similar_sg, 1):
            print(f"  {i:2}. {w:20s} → similitud: {score:.4f}")
    else:
        print(f"  ⚠️ Palabra '{word}' no encontrada en vocabulario Skip-Gram")

In [ ]:
# CELDA 10: Probar con "elecciones"
compare_models_similarity("elecciones", model_cbow, model_skipgram, topn=10)

In [ ]:
# CELDA 11: Probar con "chancay"
compare_models_similarity("chancay", model_cbow, model_skipgram, topn=10)

In [ ]:
# CELDA 12: Probar con "delmcuencia" (probable error de tipeo para "delincuencia")
# Buscar variantes
for word in ["delincuencia", "delmcuencia", "delito", "crimen"]:
    if word in model_cbow.wv:
        print(f"\n✅ Palabra encontrada: '{word}'")
        compare_models_similarity(word, model_cbow, model_skipgram, topn=8)
        break
else:
    print("⚠️ Ninguna variante encontrada. Palabras relacionadas disponibles:")
    related = [w for w in model_cbow.wv.index_to_key[:1000] if 'delin' in w or 'crim' in w or 'segur' in w]
    print(related[:20])

In [ ]:
# CELDA 13: Explorar más palabras
palabras_test = ["presidente", "gobierno", "equipo", "partido", "economia", "dolar"]

for palabra in palabras_test:
    if palabra in model_cbow.wv:
        compare_models_similarity(palabra, model_cbow, model_skipgram, topn=8)
        print("\n")

## 📊 Visualización de Embeddings en 2D

In [ ]:
# CELDA 14: Función para visualizar embeddings con t-SNE
def visualize_embeddings_tsne(model, words, title="Word Embeddings (t-SNE)", perplexity=30):
    """
    Visualiza embeddings en 2D usando t-SNE
    """
    # Filtrar palabras que existen en el vocabulario
    valid_words = [w for w in words if w in model.wv]
    
    if len(valid_words) < 2:
        print(f"⚠️ Muy pocas palabras válidas ({len(valid_words)})")
        return
    
    # Obtener vectores
    vectors = np.array([model.wv[w] for w in valid_words])
    
    # Aplicar t-SNE
    perplexity = min(perplexity, len(valid_words) - 1)
    tsne = TSNE(n_components=2, random_state=42, perplexity=perplexity)
    vectors_2d = tsne.fit_transform(vectors)
    
    # Visualizar
    plt.figure(figsize=(14, 10))
    plt.scatter(vectors_2d[:, 0], vectors_2d[:, 1], alpha=0.6, s=100, color='steelblue')
    
    # Etiquetar puntos
    for i, word in enumerate(valid_words):
        plt.annotate(word, xy=(vectors_2d[i, 0], vectors_2d[i, 1]),
                    xytext=(5, 2), textcoords='offset points',
                    fontsize=9, alpha=0.8)
    
    plt.title(title, fontsize=14, fontweight='bold')
    plt.xlabel('Dimensión 1', fontsize=12)
    plt.ylabel('Dimensión 2', fontsize=12)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    print(f"\n✅ Visualizadas {len(valid_words)} palabras")

In [ ]:
# CELDA 15: Seleccionar palabras clave de diferentes tópicos
palabras_topicos = {
    "Política": ["presidente", "congreso", "gobierno", "ministro", "elecciones", "politico", "partido"],
    "Deportes": ["equipo", "partido", "jugador", "futbol", "entrenador", "campeonato", "goles"],
    "Economía": ["economia", "dolar", "empresa", "mercado", "banco", "financiero", "comercio"],
    "Seguridad": ["policia", "delito", "seguridad", "crimen", "detenido", "investigacion"]
}

# Combinar todas las palabras
todas_palabras = []
for topico, palabras in palabras_topicos.items():
    todas_palabras.extend(palabras)

print(f"Total de palabras seleccionadas: {len(todas_palabras)}")
print(f"Palabras: {todas_palabras}")

In [ ]:
# CELDA 16: Visualizar con t-SNE - Modelo CBOW
visualize_embeddings_tsne(
    model_cbow, 
    todas_palabras, 
    title="Word Embeddings - Modelo CBOW (t-SNE)",
    perplexity=10
)

In [ ]:
# CELDA 17: Visualizar con t-SNE - Modelo Skip-Gram
visualize_embeddings_tsne(
    model_skipgram, 
    todas_palabras, 
    title="Word Embeddings - Modelo Skip-Gram (t-SNE)",
    perplexity=10
)

In [ ]:
# CELDA 18: Función para visualizar con PCA
def visualize_embeddings_pca(model, words, title="Word Embeddings (PCA)"):
    """
    Visualiza embeddings en 2D usando PCA
    """
    # Filtrar palabras válidas
    valid_words = [w for w in words if w in model.wv]
    
    if len(valid_words) < 2:
        print(f"⚠️ Muy pocas palabras válidas ({len(valid_words)})")
        return
    
    # Obtener vectores
    vectors = np.array([model.wv[w] for w in valid_words])
    
    # Aplicar PCA
    pca = PCA(n_components=2, random_state=42)
    vectors_2d = pca.fit_transform(vectors)
    
    print(f"\nVarianza explicada por PCA:")
    print(f"  PC1: {pca.explained_variance_ratio_[0]*100:.2f}%")
    print(f"  PC2: {pca.explained_variance_ratio_[1]*100:.2f}%")
    print(f"  Total: {sum(pca.explained_variance_ratio_)*100:.2f}%")
    
    # Visualizar
    plt.figure(figsize=(14, 10))
    plt.scatter(vectors_2d[:, 0], vectors_2d[:, 1], alpha=0.6, s=100, color='coral')
    
    # Etiquetar puntos
    for i, word in enumerate(valid_words):
        plt.annotate(word, xy=(vectors_2d[i, 0], vectors_2d[i, 1]),
                    xytext=(5, 2), textcoords='offset points',
                    fontsize=9, alpha=0.8)
    
    plt.title(title, fontsize=14, fontweight='bold')
    plt.xlabel(f'PC1 ({pca.explained_variance_ratio_[0]*100:.1f}%)', fontsize=12)
    plt.ylabel(f'PC2 ({pca.explained_variance_ratio_[1]*100:.1f}%)', fontsize=12)
    plt.grid(True, alpha=0.3)
    plt.axhline(y=0, color='k', linestyle='--', alpha=0.3)
    plt.axvline(x=0, color='k', linestyle='--', alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    print(f"\n✅ Visualizadas {len(valid_words)} palabras")

In [ ]:
# CELDA 19: Visualizar con PCA - Modelo CBOW
visualize_embeddings_pca(
    model_cbow, 
    todas_palabras, 
    title="Word Embeddings - Modelo CBOW (PCA)"
)

In [ ]:
# CELDA 20: Visualizar con PCA - Modelo Skip-Gram
visualize_embeddings_pca(
    model_skipgram, 
    todas_palabras, 
    title="Word Embeddings - Modelo Skip-Gram (PCA)"
)

## 📰 Visualización de Documentos por Tópico

In [ ]:
# CELDA 21: Función para obtener embedding de documento
def get_document_embedding(model, tokens):
    """
    Calcula el embedding de un documento como promedio de sus tokens
    """
    vectors = []
    for token in tokens:
        if token in model.wv:
            vectors.append(model.wv[token])
    
    if len(vectors) == 0:
        return None
    
    return np.mean(vectors, axis=0)

# Calcular embeddings para muestra de documentos
print("Calculando embeddings de documentos...")

# Seleccionar 10 documentos por categoría (total ~50 docs)
categories = ["Deportes", "Política", "Economía", "Mundo", "Espectáculos"]
sample_docs = []
sample_labels = []

for cat in categories:
    docs_cat = df[df['seccion'] == cat].head(10)
    for idx in docs_cat.index:
        tokens = sentences[idx]
        emb = get_document_embedding(model_cbow, tokens)
        if emb is not None:
            sample_docs.append(emb)
            sample_labels.append(cat)

sample_docs = np.array(sample_docs)
print(f"\n✅ {len(sample_docs)} documentos con embeddings calculados")

In [ ]:
# CELDA 22: Visualizar documentos con t-SNE
print("Aplicando t-SNE a documentos...")

tsne = TSNE(n_components=2, random_state=42, perplexity=min(30, len(sample_docs)-1))
docs_2d = tsne.fit_transform(sample_docs)

# Crear figura
plt.figure(figsize=(14, 10))

# Colores por categoría
colors = {'Deportes': 'red', 'Política': 'blue', 'Economía': 'green', 
          'Mundo': 'orange', 'Espectáculos': 'purple'}

for cat in categories:
    indices = [i for i, label in enumerate(sample_labels) if label == cat]
    if indices:
        plt.scatter(
            docs_2d[indices, 0], 
            docs_2d[indices, 1],
            c=colors.get(cat, 'gray'),
            label=cat,
            alpha=0.6,
            s=100
        )

plt.title('Noticias por Tópico en Espacio de Embeddings (t-SNE)', 
          fontsize=14, fontweight='bold')
plt.xlabel('Dimensión 1', fontsize=12)
plt.ylabel('Dimensión 2', fontsize=12)
plt.legend(loc='best', fontsize=10)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("\n✅ Visualización completada")

In [ ]:
# CELDA 23: Visualizar documentos con PCA
print("Aplicando PCA a documentos...")

pca = PCA(n_components=2, random_state=42)
docs_2d_pca = pca.fit_transform(sample_docs)

print(f"\nVarianza explicada:")
print(f"  PC1: {pca.explained_variance_ratio_[0]*100:.2f}%")
print(f"  PC2: {pca.explained_variance_ratio_[1]*100:.2f}%")
print(f"  Total: {sum(pca.explained_variance_ratio_)*100:.2f}%")

# Visualizar
plt.figure(figsize=(14, 10))

for cat in categories:
    indices = [i for i, label in enumerate(sample_labels) if label == cat]
    if indices:
        plt.scatter(
            docs_2d_pca[indices, 0], 
            docs_2d_pca[indices, 1],
            c=colors.get(cat, 'gray'),
            label=cat,
            alpha=0.6,
            s=100
        )

plt.title('Noticias por Tópico en Espacio de Embeddings (PCA)', 
          fontsize=14, fontweight='bold')
plt.xlabel(f'PC1 ({pca.explained_variance_ratio_[0]*100:.1f}%)', fontsize=12)
plt.ylabel(f'PC2 ({pca.explained_variance_ratio_[1]*100:.1f}%)', fontsize=12)
plt.legend(loc='best', fontsize=10)
plt.grid(True, alpha=0.3)
plt.axhline(y=0, color='k', linestyle='--', alpha=0.3)
plt.axvline(x=0, color='k', linestyle='--', alpha=0.3)
plt.tight_layout()
plt.show()

## 📊 Comparación CBOW vs Skip-Gram

In [ ]:
# CELDA 24: Resumen comparativo
print("=" * 80)
print("COMPARACIÓN: CBOW vs SKIP-GRAM")
print("=" * 80)

print(f"""
📊 **Características de los modelos:**

🔹 **CBOW (Continuous Bag of Words)**:
   - Vocabulario: {len(model_cbow.wv)} palabras
   - Predice: palabra central dado contexto
   - Velocidad: Más rápido de entrenar
   - Uso ideal: Palabras frecuentes, corpus grandes
   - Ventaja: Mejor para capturar el contexto general

🔹 **Skip-Gram**:
   - Vocabulario: {len(model_skipgram.wv)} palabras
   - Predice: contexto dada palabra central
   - Velocidad: Más lento de entrenar
   - Uso ideal: Palabras poco frecuentes, corpus pequeños
   - Ventaja: Mejor para capturar relaciones semánticas sutiles

💡 **Observaciones en este corpus:**
   - Ambos modelos capturan bien las relaciones semánticas
   - CBOW es más eficiente para este tamaño de corpus
   - Skip-Gram puede dar mejores resultados para palabras raras
   - La visualización muestra clusters claros por tópico
""")

In [ ]:
# CELDA 25: Guardar modelos (opcional)
# model_cbow.save("word2vec_cbow_noticias.model")
# model_skipgram.save("word2vec_skipgram_noticias.model")
# print("✅ Modelos guardados")

print("\n💾 Para guardar los modelos, descomenta las líneas arriba")

In [ ]:
# CELDA 26: Resumen final
print("=" * 80)
print("📝 RESUMEN DEL ANÁLISIS DE WORD EMBEDDINGS")
print("=" * 80)

print(f"""
✅ **Resultados del análisis:**

1. **Datos procesados**:
   - Documentos: {len(df)}
   - Sentencias procesadas: {len(sentences)}
   - Categorías: {df['seccion'].nunique()}

2. **Modelos entrenados**:
   - CBOW: {len(model_cbow.wv)} palabras en vocabulario
   - Skip-Gram: {len(model_skipgram.wv)} palabras en vocabulario
   - Dimensión de embeddings: {vector_size}

3. **Análisis realizados**:
   ✓ Búsqueda de palabras similares
   ✓ Visualización con t-SNE
   ✓ Visualización con PCA
   ✓ Mapeo de documentos por tópico

4. **Palabras probadas**:
   - 'elecciones', 'chancay', 'delincuencia'
   - Palabras políticas, deportivas, económicas

💡 **Conclusiones**:
   - Los embeddings capturan bien las relaciones semánticas
   - Los documentos se agrupan claramente por tópico
   - Ambos modelos (CBOW y Skip-Gram) son efectivos
   - La reducción dimensional (t-SNE/PCA) preserva la estructura
""")